# Dog, cat image classification problem set

* 이번 학습에서는 처음부터 끝까지 Dog, cat dataset에 대한 분류 model을 구현합니다

### [CUDA](http://pytorch.org/docs/stable/cuda.html)

* cuda를 이용하겠습니다


In [1]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
#train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


---
## Load the [Data](http://pytorch.org/docs/stable/torchvision/datasets.html)

* 아미지를 다운로드 받습니다
* 폴더별로
 - test
 - train
 - validation

 data를 받습니다. 


In [2]:
!pip install googledrivedownloader

In [3]:
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile 

#if exists("./Cat_Dog_data.tgz"):
#    !rm -rf ./Cat_Dog_data.tgz

gdd.download_file_from_google_drive(file_id='1WpY0qpe7yF5C5M52z1BMIzYVpDYiU3OV',
                                    dest_path = './Cat_Dog_data.tgz')

tf = tarfile.open("Cat_Dog_data.tgz")
tf.extractall()

## Problem 1 [20 points]: 

* Training, validation, test를 위한 dataloader, transform을 적절하게 준비해주세요
* 아래 data 준비하는 코딩을 수행하고, 아래 markdown에 준비한 과정 및 이유를 구체적으로 설명하세요
* 아래 답안 작성에 data의 구조에 대해서 설명하세요
* 코드에는 주석을 달아주세요.

In [4]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
%matplotlib inline

# Coding
train_dir = 'train'       # train 경로 설정
test_dir = 'test'         # test 경로 설정
valid_size = 0.2          # validation set으로 분할 할 비율
batch_size = 20

# 적용 변형 
train_transforms = transforms.Compose ([                                    # 이미지 리사이즈
                                transforms.RandomRotation (30),           
                                transforms.RandomResizedCrop (224), 
                                transforms.RandomHorizontalFlip (),
                                transforms.ToTensor (),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])])
test_transforms = transforms.Compose ([transforms.Resize (255),        
                                      transforms.CenterCrop (224), 
                                      transforms.ToTensor (),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder (train_dir,   
                                    transform = train_transforms)                                        
test_data = datasets.ImageFolder (test_dir, 
                                    transform = test_transforms)

num_train = len(train_data)       # train 데이터의 길이
indices = list(range(num_train))    # train 데이터 index
np.random.shuffle(indices)    # 순서 랜덤으로 바꿈 섞어서 하는 것이 random성에 좋다
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)      # DataLoder에 sampler 넣기
valid_sampler = SubsetRandomSampler(valid_idx)      # DataLoder에 sampler 넣기

# 데이터 로딩
train_loader = torch.utils.data.DataLoader (train_data, batch_size = batch_size,sampler=train_sampler)      # trainloder 생성
test_loader = torch.utils.data.DataLoader (test_data, batch_size = batch_size)    # testloder 생성
valid_loader = torch.utils.data.DataLoader(train_data, batch_size= batch_size,     # validloder 생성
    sampler=valid_sampler)


# helper function to un-normalize and display an image
def imshow(img): # 이미지를 보기위한 함수 ()
    img = img * 0.5 + 0.5  # unnormalize 
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image


 # obtain one batch of training images
dataiter = iter(train_loader) # train_loader에서 데이터를 받음
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display, tensor를 numpy로 바꿈


print(images.shape)     # 사이즈 확인


classes = ['cat','dog']

(20, 3, 224, 224)


**분석 및 설명:**transforms를 통해 데이터를 리사이즈하고 크롭한다. 크롭한 데이터를 이미지폴더에 넣고 train데이터의 길이를 가져와서 리스트에 저장하여 랜덤화 효과를 위해 순서를 섞는다. 섞인 데이터를 다시 dataloder를 통해 sampler에 넣어준다.

## Problem 2 [20 point]: Define the Network Architecture

* 구현하고자하는 network을 작성해주세요
* 아래 구현 방법과 이유를 구체적으로 설명하세요
* 코드에는 주석을 달아주세요. 
* 아래 모델을 구체적으로 설명하고, 설정 이유를 작성해주세요
* 본 과정에서는 직접 network을 구현하고, transfer learning은 활용하지 않도록 하겠습니다.

In [6]:
# 코드 작성
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (224,224,3 image tensor)
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # convolutional layer (112*112*32 tensor)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # convolutional layer (56*56*64 tensor)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (28*28*128 -> 500)
        self.fc1 = nn.Linear(128*28*28, 1000)
        # linear layer (500 -> 2)
        self.fc2 = nn.Linear(1000, 500)      # 분류하는게 2개라서
        self.fc3 = nn.Linear(500, 2)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 128*28*28)
        # add dropout layer
        x = self.dropout(x) 
        # add 1 hidden layer, relu activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # aadd 2 hidden layer, relu activation
        x = F.log_softmax(self.fc3(x),dim=1)
        return x


model = Net()
print(model)
if train_on_gpu:
    model.cuda() 

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


**분석 및 설명:** 3개의 컨볼루션 레이어를 만들고 maxpool을 통해 최대값을 뽑아낸다. 최적화과정에서 NLLLoss를 사용하기 때문에 softmax를 사용하였다. 오버피팅을 막기 위해 dropout을 넣고, relu를 적용하여 0이상의 값들은 그대로 출력을 한다. 

## Problem 3 [5 point]: Specify Loss Function

* Loss 함수와 optimizer를 구현하세요
* 선정 이유를 설명하세요


In [0]:
# 코드
import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.9)

 **설명:** softmax를 적용시키고 NLLLoss를 사용하면 CrossEntropy랑 같은효과를 낸다하여 NLLLoss를 사용하였다.
 


## Problem 4 [30 point]: Train the Network

* training loss와 validation loss를 기록하며 training을 구현하세요
* 만약 validation loss가 최소화된 모델을 저장하세요
* 코드에는 모두 주석을 포함해주세요
* training과정을 설명하고, training 결과를 분석해주세요

In [8]:
# 코드 작성
n_epochs = 35

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:        # 
            data, target = data.cuda(), target.cuda()
        # 최적화된 변수의 그라데이션을 지움
        optimizer.zero_grad()
        # forward pass: 모델에 입력을 전달하여 예측 출력을 전달한다
        output = model(data)
        # batch loss 계산
        loss = criterion(output, target)
        # backward pass: 모델 매개변수에 대한 손실 계산
        loss.backward()
        # 단일 최적화  모델에 입력을 전달하여 예측 출력을 전달한다
        output = model(data)
        # batch loss 계산
        loss = criterion(output, target)
        # backward pass: 모델 매개변수에 대한 손실 계산
        loss.backward()
        # 단일 최적화 단계 수행
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:          # train_on
            data, target = data.cuda(), target.cuda()
        # forward pass: 모델에 입력을 전달하여 예측 출력 계산
        output = model(data)
        # batch loss 계산
        loss = criterion(output, target)
        # validation loss 평균 업데이트
        valid_loss += loss.item()*data.size(0)
    
    # losses 평균 계산
    train_loss = train_loss/len(train_loader.sampler)   # train_loader 크기로 나눔
    valid_loss = valid_loss/len(valid_loader.sampler)   # valid_loader 크기로 나눔
        
    # traning / validation 상태 출력 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(       # 결과 출력
        epoch, train_loss, valid_loss))
    
    # 비교하여 validation loss가 낮은것이 저장
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_catdog.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.677333 	Validation Loss: 0.638312
Validation loss decreased (inf --> 0.638312).  Saving model ...
Epoch: 2 	Training Loss: 0.641688 	Validation Loss: 0.637168
Validation loss decreased (0.638312 --> 0.637168).  Saving model ...
Epoch: 3 	Training Loss: 0.616351 	Validation Loss: 0.595624
Validation loss decreased (0.637168 --> 0.595624).  Saving model ...
Epoch: 4 	Training Loss: 0.601661 	Validation Loss: 0.574195
Validation loss decreased (0.595624 --> 0.574195).  Saving model ...
Epoch: 5 	Training Loss: 0.588810 	Validation Loss: 0.609953
Epoch: 6 	Training Loss: 0.586397 	Validation Loss: 0.579349
Epoch: 7 	Training Loss: 0.577728 	Validation Loss: 0.557997
Validation loss decreased (0.574195 --> 0.557997).  Saving model ...
Epoch: 8 	Training Loss: 0.565429 	Validation Loss: 0.538615
Validation loss decreased (0.557997 --> 0.538615).  Saving model ...
Epoch: 9 	Training Loss: 0.553421 	Validation Loss: 0.553278
Epoch: 10 	Training Loss: 0.552695 	Valida

**분석 및 설명:**Epoch 수만큼 dataset을 사용, 앞서 설계했던 네트워크를 통해 training을 수행한다. 이후 validation loss를 계산하여 출력한다. validation loss를 비교하여 낮은것이 저장된다.

## Problem 5 [5 point] Validation Loss가 최소화된 Model가져오기

* 최소 validation loss를 갖는 model을 불러옵니다

In [9]:
# 코드 작성
model.load_state_dict(torch.load('model_catdog.pt'))        # 저장된 validation loss 가져오기

<All keys matched successfully>

---
## Problem 6 [20point]: Test the Trained Network

* Test set을 활용하여 성능 검증
* Accuracy (분류 성공률)와 test loss를 출력하세요
* 코드에는 주석을 달아주세요
* 아래 test 결과에 대해서 간단하게 설명/분석 해주세요

In [10]:
# 코드 작성
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # batch loss 계산
    loss = criterion(output, target)
    # test loss 업데이트
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))       # target의 데이터를 prediction화 해서 같은지 비교
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())   # train_on gpu가 아니면 correct_tensor를 numpy화 맞으면 cpu()로 옮긴후 numpy
    # calculate test accuracy for each object class     # 각 클래스 정확도 계산
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        
        

# average test loss
test_loss = test_loss/len(test_loader.dataset)      # test_loss를 testloader길이로 나눔
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    if class_total[i] > 0:      # class_total index i가 0보다 크면
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))    # 확률 출력
           
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))    # 0보다 작은경우 문구출력

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))          # class 토탈의 합으로 나눔




Test Loss: 0.295598

Test Accuracy of   cat: 83% (1046/1250)
Test Accuracy of   dog: 90% (1136/1250)

Test Accuracy (Overall): 87% (2182/2500)


**분석 및 설명:**validation에서는 전체 네트워크를 사용하기 때문에 dropout을 비활성화 시킬 필요가 있다. 이를 위해 model.eval를 실행을 한다. loss가 최소화된 model을 가져와서 분류가 성공한 비율을 찾는 과정이다.

---
## Problem 7: 전체적인 총평

* Data준비, Training과 validation 과정에서 성능 개선을 위해서 수행한 과정과 성공/실패 이유를 분석해주세요


**설명:**dropout을 바꿔보았고, nomalize를 추가하였다. convolution layer 역시 조절을 해보았지만 크게 유의미한 결과는 나타나지 않았다. 다만 nomalize를 추가했을때 차이가 5%정도로 나타났다. batch_size는 개선이 조금 보였다. 처음에 32로 설정했을때 계속 index 오류가 떠서 원인은 모르겠지만 사이즈를 줄이니 오류는 뜨지 않았다. 네트워크을 잘못 짜서 그런지 loss가 너무 조금씩 줄어들었다.